In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
par_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
os.chdir(par_dir)
from multilayer_model.m2_bce import BCEcVAE
from multilayer_model.m2_mse import MSEcVAE
from multilayer_loss.l2_bce import l2_bce
from multilayer_loss.l2_mse import l2_mse
from vae_earlystopping import EarlyStopping

In [3]:
x_data = np.load('./data/metal.npy')      # Target (X)
x2_data = np.load('./data/support_norm.npy')
c_data = np.load('./data/pre_re_change_temp_logconst.npy') # Condition (C)

# 데이터 분할 (Train/Val/Test)
c_train, c_temp, x_train, x_temp, x2_train, x2_temp = train_test_split(c_data, x_data,x2_data, test_size=0.4, random_state=42)
c_val, c_test, x_val, x_test, x2_val, x2_test = train_test_split(c_temp, x_temp,x2_temp, test_size=0.5, random_state=42)

# 스케일링
c_scaler, x_scaler,x2_scaler = MinMaxScaler(), MinMaxScaler(), MinMaxScaler()
c_train = c_scaler.fit_transform(c_train)
x_train = x_scaler.fit_transform(x_train)
x2_train = x2_scaler.fit_transform(x2_train)
c_val, x_val, x2_val = c_scaler.transform(c_val), x_scaler.transform(x_val), x2_scaler.transform(x2_val)
c_test, x_test, x2_test = c_scaler.transform(c_test), x_scaler.transform(x_test), x2_scaler.transform(x2_test)

# 텐서 변환
def to_tensor(arr): return torch.tensor(arr, dtype=torch.float32)
c_train, x_train, x2_train = to_tensor(c_train), to_tensor(x_train), to_tensor(x2_train)
c_val, x_val, x2_val = to_tensor(c_val), to_tensor(x_val), to_tensor(x2_val)
c_test, x_test, x2_test = to_tensor(c_test), to_tensor(x_test), to_tensor(x2_test)

train_loader = DataLoader(TensorDataset(x_train, x2_train, c_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val, x2_val, c_val), batch_size=64, shuffle=False)
test_loader = DataLoader(TensorDataset(x_test, x2_test, c_test), batch_size=64, shuffle=False)

x,x2, c = next(iter(train_loader))
x_dim = x.shape[1]
x2_dim = x2.shape[1]
c_dim = c.shape[1]
z_dim = 8
z2_dim = 8
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 3. BCE 및 MSE 모델 정의
model_bce = BCEcVAE(x_dim, x2_dim, c_dim, z_dim, z2_dim).to(device)
model_mse = MSEcVAE(x_dim, x2_dim, c_dim, z_dim, z2_dim).to(device)

In [4]:
optimizer_bce = optim.Adam(model_bce.parameters(), lr=1e-3, weight_decay=1e-5)
optimizer_mse = optim.Adam(model_mse.parameters(),lr=1e-3, weight_decay = 1e-5)
es_bce= EarlyStopping(patience=40, min_delta=1e-9)
es_mse = EarlyStopping(patience=40,min_delta=1e-9)

In [5]:
for epoch in range(1, 801):
    model_bce.train()
    t_loss = 0
    for x, x2, c in train_loader:
        x, x2, c = x.to(device), x2.to(device), c.to(device)
        optimizer_bce.zero_grad()
        logit, mu, logvar = model_bce(x, x2, c)
        loss = l2_bce(logit, x2, mu, logvar)['loss']
        loss.backward()
        optimizer_bce.step()
        t_loss += loss.item()
    
    model_bce.eval()
    v_loss = 0
    with torch.no_grad():
        for vx, vx2, vc in val_loader:
            vx, vx2, vc = vx.to(device), vx2.to(device), vc.to(device)
            v_logit, v_mu, v_logvar = model_bce(vx, vx2, vc)
            v_loss += l2_bce(v_logit, vx2, v_mu, v_logvar)['loss'].item()
    
    avg_v_loss = v_loss / len(val_loader)
    if epoch % 50 == 0: print(f"Epoch {epoch} | Val Loss: {avg_v_loss:.6f}")
    if es_bce(avg_v_loss, model_bce): break

es_bce.load_best_model(model_bce)

# --- 4. MSE 모델 학습 (수치 예측) ---
print("\n[Phase 2] Training Weighted MSEcVAE...")
model_mse = MSEcVAE(x_dim, x2_dim, c_dim, z_dim,z2_dim).to(device)
optimizer_mse = optim.Adam(model_mse.parameters(), lr=1e-3, weight_decay=1e-5)
es_mse = EarlyStopping(patience=40, min_delta=1e-9)

for epoch in range(1, 801):
    model_mse.train()
    t_loss = 0
    for x, x2, c in train_loader:
        x, x2, c = x.to(device), x2.to(device), c.to(device)
        optimizer_mse.zero_grad()
        x_hat, mu, logvar = model_mse(x, x2, c)
        loss = l2_mse(x_hat, x2, mu, logvar)['loss']
        loss.backward()
        optimizer_mse.step()
        t_loss += loss.item()

    model_mse.eval()
    v_loss = 0
    with torch.no_grad():
        for vx, vx2, vc in val_loader:
            vx, vx2, vc = vx.to(device), vx2.to(device), vc.to(device)
            vh, vm, vl = model_mse(vx, vx2, vc)
            v_loss += l2_mse(vh, vx2, vm, vl)['loss'].item()
    
    avg_v_loss = v_loss / len(val_loader)
    if epoch % 50 == 0: print(f"Epoch {epoch} | Val Loss: {avg_v_loss:.6f}")
    if es_mse(avg_v_loss, model_mse): break

es_mse.load_best_model(model_mse)

# --- 5. 최종 평가 (Hurdle: BCE Prob * MSE Value) ---
model_bce.eval()
model_mse.eval()

all_pred = []
all_true = []
all_bce =[]
all_prob=[]
all_binary=[]
all_ct=[]
all_xt=[]
with torch.no_grad():
    for xt, xt2, ct in test_loader:
        xt, xt2, ct = xt.to(device), xt2.to(device), ct.to(device)
        
        # BCE 확률 (P)
        b_logit, _, _ = model_bce(xt, xt2, ct)
        b_prob = torch.sigmoid(b_logit)
        b_binary = (b_prob > 0.5).float()
        
        # MSE 수치 (V)
        m_val, _, _ = model_mse(xt, xt2, ct)
        
        # 결합 (P * V)
        combined_s = b_prob * m_val
        all_bce.append(b_logit.cpu().numpy())
        all_prob.append(b_prob.cpu().numpy())
        all_xt.append(xt.cpu().numpy())
        all_binary.append(b_binary.cpu().numpy())
        all_pred.append(combined_s.cpu().numpy())
        all_ct.append(ct.cpu().numpy())
        all_true.append(xt.cpu().numpy())


EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
Epoch 50 | Val Loss: 0.306900
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 

TypeError: MSEcVAE.__init__() missing 1 required positional argument: 'z2_dim'

In [ ]:
temp = all_xt[10:13]   # (3, 64, 23)
all_xt1 = [x[0] for x in temp]   # (3, 23)
all_xt1

[array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.26193333, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ], dtype=float32),
 array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0.2, 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ], dtype=float32),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.16666667, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ], dtype=float32)]

In [ ]:
temp = all_binary[10:13]   # (3, 64, 23)
all_binary1 = [x[0] for x in temp]   # (3, 23)
all_binary1

[array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], dtype=float32)]

In [ ]:
temp = all_bce[10:13]   # (3, 64, 23)
all_bce1 = [x[0] for x in temp]   # (3, 23)
all_bce1

[array([ -51.685028,  -40.763535,  -61.101658,  -52.48787 ,  -51.989918,
         -82.25244 ,  -69.41989 ,   54.29839 ,  -16.66396 ,  -26.28009 ,
         -23.823023,  -76.31291 ,  -55.256588,  -15.594534,  -38.789406,
         -14.668018,  -17.138811, -103.80364 , -103.28685 , -115.01558 ,
         -88.02157 ,  -90.67404 ,  -39.419243], dtype=float32),
 array([ -6.9845963, -69.30183  , -38.65771  , -22.129139 ,  -8.569847 ,
        -39.320724 , -65.04556  ,  -4.897331 ,  19.09697  ,  24.217527 ,
        -16.810646 , -28.46956  ,  -4.8526807,  -5.6024184, -55.778748 ,
        -23.352978 , -32.24361  , -26.841587 , -32.54918  , -40.27703  ,
        -64.00558  , -58.978966 , -23.312267 ], dtype=float32),
 array([-34.249954, -26.432896, -21.535563, -56.194866, -17.497656,
        -32.775204, -27.216883,  61.754402, -13.483417, -29.450457,
        -27.21092 , -71.34864 , -42.929962,  -8.699131,  -9.995949,
        -29.92531 , -24.207453, -71.087746, -72.977516, -84.51844 ,
        -58.6787

In [ ]:
temp = all_prob[10:13]   # (3, 64, 23)
all_prob1 = [x[0] for x in temp]   # (3, 23)
all_prob1

[array([3.5766587e-23, 1.9798026e-18, 2.9099609e-27, 1.6025369e-23,
        2.6367294e-23, 1.8976607e-36, 7.1010772e-31, 1.0000000e+00,
        5.7934113e-08, 3.8610178e-12, 4.5060150e-11, 7.2064843e-34,
        1.0054691e-24, 1.6880290e-07, 1.4255258e-17, 4.2634423e-07,
        3.6033704e-08, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        5.9254130e-39, 0.0000000e+00, 7.5934709e-18], dtype=float32),
 array([9.2518033e-04, 7.9909645e-31, 1.6261830e-17, 2.4515293e-10,
        1.8970566e-04, 8.3796589e-18, 5.6372987e-29, 7.4111475e-03,
        1.0000000e+00, 1.0000000e+00, 5.0029847e-08, 4.3234150e-13,
        7.7469368e-03, 3.6753733e-03, 5.9648253e-25, 7.2099229e-11,
        9.9260835e-15, 2.2021497e-12, 7.3125887e-15, 3.2203843e-18,
        1.5948911e-28, 2.4308638e-26, 7.5094986e-11], dtype=float32),
 array([1.3348543e-15, 3.3138991e-12, 4.4383758e-10, 3.9344110e-25,
        2.5168921e-08, 5.8332410e-15, 1.5130664e-12, 1.0000000e+00,
        1.3938819e-06, 1.6211723e-13, 1.5221

In [ ]:
temp = all_ct[10:13]   # (3, 64, 23)
all_ct1 = [x[0] for x in temp]   # (3, 23)
all_ct1

[array([0.63492066, 0.31034482, 0.7       , 0.04166667, 1.        ,
        0.6363636 , 0.5       , 0.16424474, 0.16424474, 0.21823941,
        0.6295575 , 0.65656567, 0.00890407], dtype=float32),
 array([3.1746033e-01, 1.7241380e-01, 8.0000001e-01, 8.3333336e-02,
        5.0000000e-01, 8.1818181e-01, 4.4589600e-01, 1.0889195e-02,
        9.1697183e-03, 3.0902890e-02, 6.0143358e-01, 2.6262626e-01,
        1.7889979e-04], dtype=float32),
 array([7.9365081e-01, 1.0344828e-01, 7.5000000e-01, 1.2500000e-01,
        1.0000000e+00, 8.1818181e-01, 5.0000000e-01, 1.6424474e-01,
        1.6424474e-01, 2.1823941e-01, 5.9597725e-01, 1.3131313e-01,
        9.1227191e-04], dtype=float32)]

In [ ]:
# Inverse Scaling 및 최종 R2 점수
y_pred_inv = x_scaler.inverse_transform(np.vstack(all_pred))
y_true_inv = x_scaler.inverse_transform(np.vstack(all_true))

final_score = r2_score(y_true_inv.flatten(), y_pred_inv.flatten())
print(f"\nFinal Combined R2 Score: {final_score:.4f}")


Final Combined R2 Score: 0.9002


In [ ]:
x_true_inv = x_scaler.inverse_transform(np.vstack(all_xt))

In [ ]:
c_true_inv = c_scaler.inverse_transform(np.vstack(all_ct))

In [ ]:
temp = x_true_inv[10:13]
all_xt = [x for x in temp]
all_xt

[array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 5.275, 0.   ,
        0.   , 1.   , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   ], dtype=float32),
 array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 6.1, 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ], dtype=float32),
 array([ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , 14.45, 10.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ], dtype=float32)]

In [ ]:
temp = c_true_inv[10:13]   # (3, 64, 23)
all_ct1 = [x for x in temp]   # (3, 23)
all_ct1

[array([6.0000000e+02, 6.0000000e+00, 5.5000000e+02, 3.0000000e+00,
        1.0000000e+02, 6.0000000e+02, 0.0000000e+00, 4.4186845e-01,
        4.4186845e-01, 9.8095423e-01, 4.1108742e+00, 3.8000000e+01,
        9.0582526e-01], dtype=float32),
 array([550.       ,   3.       , 700.       ,   2.       ,  50.       ,
        800.       ,   0.       ,   2.08156  ,   2.08156  ,   3.0934486,
          5.1984973,  78.       ,  42.17963  ], dtype=float32),
 array([700.00006  ,   2.       , 700.       ,   4.       , 100.       ,
        700.       ,   0.       ,   0.9807668,   0.9807668,   1.4662602,
          5.1357985,  70.       ,   5.692583 ], dtype=float32)]

In [ ]:
import numpy as np

all_ct1 = np.array(all_ct1)

all_ct1[:,7:11] = np.expm1(all_ct1[:,7:11])
all_ct1[:,6] = np.exp(all_ct1[:,6]) #6번째

In [ ]:
all_ct1

array([[6.0000000e+02, 6.0000000e+00, 5.5000000e+02, 3.0000000e+00,
        1.0000000e+02, 6.0000000e+02, 1.0000000e+00, 5.5561113e-01,
        5.5561113e-01, 1.6670001e+00, 6.0000019e+01, 3.8000000e+01,
        9.0582526e-01],
       [5.5000000e+02, 3.0000000e+00, 7.0000000e+02, 2.0000000e+00,
        5.0000000e+01, 8.0000000e+02, 1.0000000e+00, 7.0169649e+00,
        7.0169649e+00, 2.1052999e+01, 1.8000005e+02, 7.8000000e+01,
        4.2179630e+01],
       [7.0000006e+02, 2.0000000e+00, 7.0000000e+02, 4.0000000e+00,
        1.0000000e+02, 7.0000000e+02, 1.0000000e+00, 1.6665001e+00,
        1.6665001e+00, 3.3330002e+00, 1.6900000e+02, 7.0000000e+01,
        5.6925831e+00]], dtype=float32)